In [4]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

small_image = cv2.imread("./Images3/img_0017_aligned_small.jpg")
big_image = cv2.imread("./Images3/map17_small.jpg")

def imageFiltration(small_image, big_image):
    small_image = cv2.cvtColor(small_image, cv2.COLOR_BGR2GRAY)
    big_image = cv2.cvtColor(big_image, cv2.COLOR_BGR2GRAY)
    bilateral_result = list()
    gaussian_result = list()
    for index, elem in enumerate([small_image, big_image], 0):
        bilateral_filter = cv2.bilateralFilter(elem, 9, 75, 75)
        gaussian_filter = cv2.GaussianBlur(bilateral_filter, (5, 5), 3)
        gaussian_result.append(gaussian_filter)
    #         cv2.imshow("Bilateral" + str(index), bilateral_filter)
    #         cv2.imshow("Gaussian" + str(index), gaussian_filter)
    return gaussian_result

In [ ]:
def inside(r, q):
    rx, ry, rw, rh = r
    qx, qy, qw, qh = q
    return rx > qx and ry > qy and rx + rw < qx + qw and ry + rh < qy + qh

def hogMatching(small_image, big_image):
    winSize = (256, 256)
    cellSize = (16, 16)
    blockSize = (32, 32)
    blockStride = (16, 16)
    gaussian_r = imageFiltration(small_image, big_image)
    small, big = gaussian_r[0], gaussian_r[1]

    hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, 9)

    feature_small = hog.compute(small)
    hog.setSVMDetector(feature_small)

    rectPoints, weights = hog.detect(big)
    rectPoints = np.array([[x, y, winSize[0], winSize[1]] for (x, y) in rectPoints])
    found_filtered = list()

    for ri, r in enumerate(rectPoints):
        for qi, q in enumerate(rectPoints):
            if ri != qi and inside(r, q):
                break
        else:
            found_filtered.append(r)


    for iter,(xA,yA,xB,yB) in found_filtered:
        cv2.rectangle(big,(xA, yA), (xB, yB), (0,255,0), 1)

    cv2.imshow("Big", big)
        
hogMatching(small_image, big_image)
cv2.waitKey(0)

In [9]:
import scipy.signal as sig

kernel_x = np.array([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]])
print(kernel_x)
origin_x = np.array([[1, 2, 2],[1 ,1, 0],[2, 4, 1]])
print(origin_x)
G_x = sig.convolve2d(origin_x, kernel_x, mode='same') 
print(G_x)
result = x * kernel_x


[[-1  0  1]
 [-2  0  2]
 [-1  0  1]]
[[1 2 2]
 [1 1 0]
 [2 4 1]]
[[-5 -1  5]
 [-8  2  8]
 [-9  3  9]]
[-5  0  3]
